First import all the modules

In [56]:
from sshtunnel import SSHTunnelForwarder
import psycopg2
import time
import pandas as pd
import pandas.io.sql as sqlio

Below are the different SQL's to run

The credentials and config to connect to Odoo

In [57]:
REMOTE_HOST = "172.26.55.9"
REMOTE_SSH_PORT = 22
REMOTE_USERNAME = "yalp"
REMOTE_PASSWORD = "eenai9oB"
REMOTE_DB = "Yalp"
REMOTE_DB_USERNAME = "dashboard"
REMOTE_DB_PASSWORD = "OZptKsf2TcQn5ef4dsiRYucGjKlO6SF6"
PORT=5432

This creates an SSH tunel straight ot the Odoo server

Creating a connection to the Odoo Database.
<BR>Be careful!! The connection need to be closed manually afterwards (conn.close())

In [107]:
tunnel = SSHTunnelForwarder(
         (REMOTE_HOST, REMOTE_SSH_PORT),
         ssh_username=REMOTE_USERNAME,
         ssh_password=REMOTE_PASSWORD,
         ssh_pkey="/Users/wesselhof/.ssh/id_rsa",
         ssh_private_key_password="imacje",
         remote_bind_address=('localhost', PORT),
         local_bind_address=('localhost', PORT)
         )
tunnel.start()
conn = psycopg2.connect(
    host="localhost",
    database=REMOTE_DB,
    user=REMOTE_DB_USERNAME,
    password=REMOTE_DB_PASSWORD
    )

2021-08-19 22:04:30,165| ERROR   | Password is required for key /Users/wesselhof/.ssh/id_rsa


Finally the important part.
<BR> Here you write down the query and pass it to a dataframe when nessecary.

In [108]:
sql = "SELECT sale_order_line.id, DATE(sale_order.confirmation_date) AS confirmation_date, sale_order_line.order_id, sale_order.name, product_template.default_code, product_uom_qty, price_unit, price_subtotal FROM sale_order_line JOIN product_product ON sale_order_line.product_id = product_product.id JOIN product_template ON product_product.product_tmpl_id = product_template.id JOIN sale_order ON sale_order_line.order_id = sale_order.id WHERE sale_order.confirmation_date > '2021-01-01 00:00:00';"
df = sqlio.read_sql_query(sql, conn)



As said. IMPORTANT! Close the connection and the tunnel

In [109]:
conn.close()
if conn.closed:
    print("Connection closed")
tunnel.stop()

Connection closed


In [110]:
df


,id,confirmation_date,order_id,name,default_code,product_uom_qty,price_unit,price_subtotal
0,164282,2021-06-24,11733,L20211217-1,701375SP,8.0,70.0,560.0
1,164283,2021-06-24,11733,L20211217-1,701380SP,2.0,290.0,580.0
2,164284,2021-06-24,11733,L20211217-1,701606,1.0,199.0,199.0
3,164285,2021-06-24,11733,L20211217-1,003880SP,1.0,250.0,250.0
4,154058,2021-04-07,11085,L20210643-1,RT1245-10,1.0,92.0,92.0
...,...,...,...,...,...,...,...,...
4549,164278,2021-06-24,11733,L20211217-1,911163,1.0,35.0,35.0
4550,164281,2021-06-24,11733,L20211217-1,701055SP,2.0,400.0,800.0
4551,168014,2021-07-01,11970,L20201920-3,8712,1.0,5627.5,5627.5
4552,146317,2021-02-15,10659,YI20210017-1,US175533,1.0,1685.0,1685.0


In [118]:
sort_df = df.sort_values(by='confirmation_date')
sort_df

,id,confirmation_date,order_id,name,default_code,product_uom_qty,price_unit,price_subtotal
1623,140707,2021-01-04,10319,L20202007-2,8801,1.0,382.0,382.0
1322,139515,2021-01-04,10247,L20201257-2,8811,1.0,75.0,75.0
2355,140686,2021-01-04,10318,Y20210245-1,YA3531D2,1.0,0.0,0.0
1323,141045,2021-01-04,10247,L20201257-2,None,0.0,24712.5,0.0
1617,140699,2021-01-04,10319,L20202007-2,8801,1.0,4563.0,4563.0
...,...,...,...,...,...,...,...,...
431,174105,2021-08-19,12340,L20211745-1,8802,1.0,0.0,0.0
430,174104,2021-08-19,12340,L20211745-1,8801,1.0,0.0,0.0
269,174102,2021-08-19,12340,L20211745-1,911101,1.0,40.0,40.0
425,174101,2021-08-19,12340,L20211745-1,931102,1.0,106.5,106.5
